### Notebook used to compare the clusters found in LCTA to the groups found using the T2DM and prediabetes thresholds for fasting glucose

In [2]:
using CairoMakie
using CSV 
using DataFrames
using RData
using Combinatorics
using StatsBase  

In [3]:
file_path_LCTA = "C:/Users/20192809/OneDrive - TU Eindhoven/TUe/Thesis/R shenanigans/Vehpi data/plot_data.RData"

data_LCTA = load(file_path_LCTA)["plot_data"]

# Extract unique rows based on the 'Simulation' column
unique_simulations_LCTA = unique(data_LCTA, [:Simulation])
clusters_LCTA = unique_simulations_LCTA[:, :Cluster];

In [12]:
# load in data
file_path_simulation = "C:/Users/20192809/OneDrive - TU Eindhoven/TUe/Thesis/Synthetic data tests/Vehpi data/simulation_data.csv"

data_simulation = CSV.File(file_path_simulation) |> DataFrame
# Convert all time columns to Float64, handling comma decimal separators
for col in names(data_simulation)[3:end]  # Skip "simulation" and "State Variable"
    data_simulation[!, col] .= parse.(Float64, replace.(data_simulation[!, col], "," => "."))
end

# Filter for glucose (State Variable = 0) and insulin (State Variable = 1)
glucose = filter(row -> row[:"State Variable"] == 0, data_simulation)
insulin = filter(row -> row[:"State Variable"] == 1, data_simulation)

# Convert glucose values to mmol/L (mg/dL ÷ 18.018)
glucose[:, 3:end] .= glucose[:, 3:end] ./ 18.018  # Element-wise division for all time columns


Row,Time_0,Time_5,Time_10,Time_15,Time_20,Time_25,Time_30,Time_35,Time_40,Time_45,Time_50,Time_55,Time_60,Time_65,Time_70,Time_75,Time_80,Time_85,Time_90,Time_95,Time_100,Time_105,Time_110,Time_115,Time_120,Time_125,Time_130,Time_135,Time_140,Time_145,Time_150,Time_155,Time_160,Time_165,Time_170,Time_175,Time_180,Time_185,Time_190,Time_195,Time_200,Time_205,Time_210,Time_215,Time_220,Time_225,Time_230,Time_235,Time_240,Time_245,Time_250,Time_255,Time_260,Time_265,Time_270,Time_275,Time_280,Time_285,Time_290,Time_295,Time_300,Time_305,Time_310,Time_315,Time_320,Time_325,Time_330,Time_335,Time_340,Time_345,Time_350,Time_355,Time_360,Time_365,Time_370,Time_375,Time_380,Time_385,Time_390,Time_395,Time_400,Time_405,Time_410,Time_415,Time_420,Time_425,Time_430,Time_435,Time_440,Time_445,Time_450,Time_455,Time_460,Time_465,Time_470,Time_475,Time_480,Time_485,Time_490,Time_495,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,4.55467,4.55465,4.55464,4.55462,4.55461,4.5546,4.55458,4.55457,4.55456,4.55455,4.55453,4.55452,4.55451,4.5545,4.55448,4.55447,4.55446,4.55445,4.55444,4.55442,4.55441,4.5544,4.55439,4.55437,4.55436,4.55435,4.55434,4.55433,4.55432,4.55431,4.5543,4.55429,4.55428,4.55427,4.55426,4.55425,4.55424,4.55423,4.55422,4.55421,4.5542,4.55419,4.55418,4.55417,4.55416,4.55415,4.55415,4.55414,4.55413,4.55412,4.55411,4.5541,4.55409,4.55409,4.55408,4.55407,4.55406,4.55405,4.55404,4.55403,4.55403,4.55403,4.55793,4.56446,4.57135,4.57852,4.58595,4.59363,4.60155,4.60972,4.61813,4.62678,4.63568,4.64482,4.6542,4.66382,4.67365,4.68372,4.69401,4.70449,4.71519,4.72608,4.73716,4.74844,4.75987,4.77146,4.78321,4.79509,4.80712,4.81927,4.83147,4.84374,4.85606,4.86844,4.88084,4.89326,4.90564,4.91799,4.9303,4.94254,⋯
2,4.94635,4.94633,4.94631,4.94628,4.94626,4.94624,4.94622,4.9462,4.94618,4.94616,4.94615,4.94613,4.94612,4.94611,4.94609,4.94608,4.94606,4.94605,4.94604,4.94602,4.94601,4.94599,4.94598,4.94597,4.94595,4.94594,4.94592,4.94591,4.9459,4.94588,4.94587,4.94585,4.94584,4.94582,4.94581,4.9458,4.94578,4.94577,4.94575,4.94574,4.94573,4.94571,4.9457,4.94568,4.94567,4.94566,4.94564,4.94563,4.94561,4.9456,4.94559,4.94557,4.94556,4.94555,4.94554,4.94552,4.94551,4.9455,4.94548,4.94547,4.94546,4.94546,4.94845,4.95338,4.9587,4.96431,4.97022,4.9764,4.98285,4.98955,4.9965,5.0037,5.01115,5.01885,5.02676,5.03491,5.04329,5.05191,5.0607,5.06968,5.07887,5.08827,5.09783,5.10755,5.11742,5.12746,5.13763,5.14793,5.15835,5.16891,5.17957,5.19033,5.20117,5.2121,5.22309,5.23415,5.24527,5.25644,5.26764,5.27888,⋯
3,5.54058,5.54061,5.54065,5.54069,5.54073,5.54077,5.54081,5.54084,5.54088,5.54092,5.54095,5.54099,5.54103,5.54106,5.5411,5.54114,5.54117,5.54121,5.54124,5.54128,5.54132,5.54135,5.54139,5.54142,5.54145,5.54149,5.54152,5.54155,5.54158,5.54161,5.54164,5.54167,5.5417,5.54173,5.54177,5.5418,5.54183,5.54186,5.54189,5.54192,5.54195,5.54198,5.54202,5.54205,5.54208,5.54211,5.54214,5.54217,5.5422,5.54223,5.54225,5.54228,5.54231,5.54234,5.54237,5.54239,5.54242,5.54245,5.54248,5.5425,5.54253,5.54253,5.54694,5.55378,5.561,5.56853,5.57635,5.58445,5.5928,5.60137,5.6102,5.61926,5.62852,5.63798,5.64766,5.65751,5.66752,5.67771,5.68807,5.69856,5.7092,5.71997,5.73086,5.74185,5.75294,5.76417,5.77549,5.7869,5.79838,5.80993,5.82153,5.8331

In [5]:
# Define cutoffs
T2DM_cutoff = 6.9

# Extract only the glucose values (ignoring first two columns)
time_columns = names(glucose)[3:end]  # All time columns

# Define function to classify each simulation and return a vector
function classify_glucose(row)
    glucose_values = row[time_columns] |> collect  # Convert row to a vector
    max_glucose = maximum(glucose_values)
    final_glucose = glucose_values[end]
    
    # Check for "Intervention-responsive" group
    went_above = false
    went_below_after_exceed = false
    
    for val in glucose_values
        if val > T2DM_cutoff
            went_above = true
        end
        if went_above && val < T2DM_cutoff
            went_below_after_exceed = true
            break  # No need to check further once condition is met
        end
    end
    
    # Determine group classification
    if max_glucose <= T2DM_cutoff
        return "Non-diabetic"
    elseif went_above && went_below_after_exceed
        return "Intervention-responsive"
    else
        return "Diabetic"
    end
end

# Apply function to classify and return a vector of classifications
glucose_groups = classify_glucose.(eachrow(glucose));


In [6]:
# Direct cluster-to-group mapping (example provided by you)
cluster_to_group_mapping = Dict(2 => "Non-diabetic", 1 => "Intervention-responsive", 3 => "Diabetic")

# Initialize dictionaries to count accurate and misclassifications for each group
accurate_classifications = Dict("Non-diabetic" => 0, "Intervention-responsive" => 0, "Diabetic" => 0)
misclassifications = Dict("Non-diabetic" => 0, "Intervention-responsive" => 0, "Diabetic" => 0)

# Count accurate classifications and misclassifications
for (cluster, group) in zip(clusters_LCTA, glucose_groups)
    if cluster_to_group_mapping[cluster] == group
        accurate_classifications[group] += 1  # Correct classification
    else
        misclassifications[group] += 1  # Incorrect classification
    end
end

# Display the counts of accurate and misclassified classifications per group
println("Accurate classifications: ", accurate_classifications)
println("Misclassifications: ", misclassifications)


Accurate classifications: Dict("Non-diabetic" => 674, "Intervention-responsive" => 100, "Diabetic" => 195)
Misclassifications: Dict("Non-diabetic" => 226, "Intervention-responsive" => 4, "Diabetic" => 85)


In [7]:
# Direct cluster-to-group mapping (example provided by you)
cluster_to_group_mapping = Dict(2 => "Non-diabetic", 1 => "Intervention-responsive", 3 => "Diabetic")

# Initialize confusion matrix (rows: actual class, columns: predicted class)
conf_matrix = zeros(Int, 3, 3)

# Initialize dictionaries to count accurate and misclassifications for each group
accurate_classifications = Dict("Non-diabetic" => 0, "Intervention-responsive" => 0, "Diabetic" => 0)
misclassifications = Dict("Non-diabetic" => 0, "Intervention-responsive" => 0, "Diabetic" => 0)

# Create a mapping to index the rows and columns of the confusion matrix
group_to_index = Dict("Non-diabetic" => 1, "Intervention-responsive" => 2, "Diabetic" => 3)

# Count accurate classifications and misclassifications
for (cluster, group) in zip(clusters_LCTA, glucose_groups)
    predicted_group = cluster_to_group_mapping[cluster]
    
    # Get row and column indices for the confusion matrix
    row = group_to_index[group]
    col = group_to_index[predicted_group]
    
    if predicted_group == group
        accurate_classifications[group] += 1  # Correct classification
        conf_matrix[row, col] += 1  # Increment the diagonal for accurate predictions
    else
        misclassifications[group] += 1  # Incorrect classification
        conf_matrix[row, col] += 1  # Increment the misclassified group in the matrix
    end
end

# Display confusion matrix
println("Confusion Matrix:")
println("                   Predicted")
println("Actual   Non-diabetic  Intervention-responsive  Diabetic")
for i in 1:3
    println(["Non-diabetic", "Intervention-responsive", "Diabetic"][i], "   ", conf_matrix[i, 1], "                ", conf_matrix[i, 2], "                   ", conf_matrix[i, 3])
end

# Display the counts of accurate and misclassified classifications per group
println("\nAccurate classifications: ", accurate_classifications)
println("Misclassifications: ", misclassifications)


Confusion Matrix:
                   Predicted
Actual   Non-diabetic  Intervention-responsive  Diabetic
Non-diabetic   674                226                   0
Intervention-responsive   0                100                   4
Diabetic   0                85                   195

Accurate classifications: Dict("Non-diabetic" => 674, "Intervention-responsive" => 100, "Diabetic" => 195)
Misclassifications: Dict("Non-diabetic" => 226, "Intervention-responsive" => 4, "Diabetic" => 85)


In [14]:
glucose[:,:Cluster] = glucose_groups
insulin[:,:Cluster] = glucose_groups

1284-element Vector{String}:
 "Non-diabetic"
 "Diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Diabetic"
 ⋮
 "Non-diabetic"
 "Non-diabetic"
 "Non-diabetic"
 "Diabetic"
 "Diabetic"
 "Non-diabetic"
 "Intervention-responsive"
 "Diabetic"
 "Non-diabetic"

In [38]:
using CairoMakie

# Define colours using Makie.wong_colors()
colors = Makie.wong_colors()

group_colors = Dict(
    1 => colors[2],  
    2 => colors[3],  
    3 => colors[1]   
)

# Create figure with a wider resolution
fig = Figure(resolution = (1400, 800))

# Glucose plot (left)
ax1 = Axis(fig[1, 1], 
    xlabel = "τ (days)", 
    ylabel = "Fasting glucose (mmol/L)", 
    xlabelsize = 24, ylabelsize = 24, xticklabelsize = 14, yticklabelsize = 14, 
    xlabelfont = :bold, ylabelfont = :bold
)

# Insulin plot (right)
ax2 = Axis(fig[1, 2], 
    xlabel = "τ (days)", 
    ylabel = "Fasting insulin (mmol/L)", 
    xlabelsize = 24, ylabelsize = 24, xticklabelsize = 14, yticklabelsize = 14, 
    xlabelfont = :bold, ylabelfont = :bold
)

# Prepare legend elements (starting with thresholds)
legend_elements = [
    LineElement(color=:red, linestyle=:dot, linewidth=3),
    LineElement(color=:black, linestyle=:dot, linewidth=3)
]

# Add cluster colours to the legend
for cluster_id in 1:3  # Explicitly loop over clusters 1, 2, 3
    pushfirst!(legend_elements, LineElement(color=group_colors[cluster_id], linewidth=4))
end

# Group by the converted group (integers)
for group in 1:3  # Explicitly loop over the groups 1, 2, 3
    group_color = group_colors[group]  # Assign colour for the current group

    # Extract glucose and insulin data for the current group
    glucose_data = glucose[glucose_groups .== group, :]
    insulin_data = insulin[insulin_groups .== group, :]

    # Reshape data to get Time_X as the x-axis and corresponding glucose/insulin values as the y-axis
    time_points = [string("Time_", i) for i in 0:5:3600]  # Time columns from 0 to 3600 in 5-minute steps
    glucose_values = Array(glucose_data[:, time_points])  # Extract glucose values for the group
    insulin_values = Array(insulin_data[:, time_points])  # Extract insulin values for the group

    # Plot glucose data
    for i in 1:size(glucose_values, 1)
        lines!(ax1, 0:5:3600, glucose_values[i, :], color = group_color, alpha = 0.5)
    end
    
    # Plot insulin data
    for i in 1:size(insulin_values, 1)
        lines!(ax2, 0:5:3600, insulin_values[i, :], color = group_color, alpha = 0.5)
    end
end

# Add horizontal dotted lines for glucose thresholds
hlines!(ax1, [5.6], linestyle=:dot, linewidth=4, color=:red)
hlines!(ax1, [6.9], linestyle=:dot, linewidth=4, color=:black)

# Add the legend to the figure with labels for the groups and thresholds
Legend(fig[1, 3], 
    legend_elements, 
    ["Remission", "Diabetic", "Non-Diabetic", "Prediabetes threshold", "T2DM threshold"], 
    labelsize = 20, labelfont = :bold)

# Display figure
fig

# Save figure
save("./Defined_groups_Vehpi_data.png", fig)


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\20192809\.julia\packages\Makie\VRavR\src\scenes.jl:220


CairoMakie.Screen{IMAGE}


In [29]:
colors = Makie.wong_colors()

In [23]:
group_colors[1]